In [1]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib
%matplotlib inline
plt.style.use('ggplot')

# display
from IPython.display import display

# autoreload
%load_ext autoreload
%autoreload 2

# warnings
import warnings
warnings.filterwarnings('ignore')

# fix random seed
from numpy.random import seed as set_random_seed
set_random_seed(42)

# explainability
# import shap, lime, eli5
# shap.initjs()

#mlflow
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")


#streamlit - delteuse
import streamlit as st
def cache(f):
    return f
st.cache = cache

## util functions
def eval_print(obj_str):
    import inspect
    frame = inspect.currentframe()
    try:
        locals().update(frame.f_back.f_locals)
        x = eval(obj_str)
        print(f'{obj_str}: {x}')
    finally:
        del frame

def findFiles(path): 
    import glob
    return glob.glob(path)

In C:\Alon\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Alon\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Alon\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Alon\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Alon\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rc

In [2]:
from utils import read_chat
chat = read_chat()

In [3]:
chat

,datetime,name,text
0,2018-05-10 13:45:00,Alon Wolf 🐺,איזה אתרים קדושים יש בירושלים\r
1,2018-05-10 13:45:00,Alon Wolf 🐺,לפי מקומות - תל-אביב - 'משימה שקשורה לים'\r\nי...
2,2018-05-10 13:46:00,Alon Wolf 🐺,לדוגמא 👆🏼\r
3,2018-05-10 13:47:00,גורגמל ״מצחיקונת״ 🍠,זה מה שאני אמרתי!\r
4,2018-05-10 13:47:00,גורגמל ״מצחיקונת״ 🍠,תעשה בירושלים את הפלאפל\r
...,...,...,...
39994,2020-11-20 07:46:00,Alon Wolf 🐺,זה * בול * מה שרציתי שיהיה לנו\r
39995,2020-11-20 07:46:00,Alon Wolf 🐺,<המדיה לא נכללה>\r
39996,2020-11-20 07:47:00,Alon Wolf 🐺,איזה כיף (:\r
39997,2020-11-20 07:47:00,Alon Wolf 🐺,תאמרי לה לשים אותם גבוה גבוה\r


In [4]:
import re
from datetime import datetime
import emoji

In [5]:

MEDIA_TOKEN = 'MEDIA'
URL_TOKEN = 'URL'
TOKENS = [MEDIA_TOKEN, URL_TOKEN]

def preprocess_text(text):
    text = re.sub('<המדיה לא נכללה>', MEDIA_TOKEN, text)
    text = re.sub('https?://\S+|www\.\S+', URL_TOKEN, text)
    return text[:-1] # removes end
text = chat['text'].apply(preprocess_text)

In [6]:
text

0                            איזה אתרים קדושים יש בירושלים
1        לפי מקומות - תל-אביב - 'משימה שקשורה לים'\r\nי...
2                                                לדוגמא 👆🏼
3                                        זה מה שאני אמרתי!
4                                  תעשה בירושלים את הפלאפל
                               ...                        
39994                       זה * בול * מה שרציתי שיהיה לנו
39995                                                MEDIA
39996                                          איזה כיף (:
39997                         תאמרי לה לשים אותם גבוה גבוה
39998                                                MEDIA
Name: text, Length: 39999, dtype: object

In [202]:
from extras.nlp.hebtokenizer import tokenize as nlp_tokenize

START_TOKEN = '###'
END_TOKEN = '$$$'

def tokenize_text(txt, max_len = 100):
    tok = [word for lng, word in nlp_tokenize(txt)]
    tok = [START_TOKEN] + tok[:(max_len - 2)] + [END_TOKEN]
    return tok
max_len = 10
tokenized = text.apply(tokenize_text)

In [199]:
tokenized

0            [###, איזה, אתרים, קדושים, יש, בירושלים, $$$]
1        [###, לפי, מקומות, -, תל-אביב, -, ', משימה, שק...
2                                   [###, לדוגמא, 👆🏼, $$$]
3                       [###, זה, מה, שאני, אמרתי, !, $$$]
4                   [###, תעשה, בירושלים, את, הפלאפל, $$$]
                               ...                        
39994    [###, זה, * , בול, * , מה, שרציתי, שיהיה, לנו,...
39995                                    [###, MEDIA, $$$]
39996                          [###, איזה, כיף, (, :, $$$]
39997        [###, תאמרי, לה, לשים, אותם, גבוה, גבוה, $$$]
39998                                    [###, MEDIA, $$$]
Name: text, Length: 39999, dtype: object

In [9]:
TEXT = text
TOKENIZED = tokenized.apply(lambda l : ' '.join(l))

In [ ]:
from utils import read_data, save_data

In [215]:
from apps.modeling.model import Model
from apps.modeling.dataset import Dataset
from apps.modeling.train import train
from extras.nlp.tokenizer import tokenize_text
text = chat['text']
tokenized_text = text.apply(tokenize_text)
model = Model(Dataset(tokenized_text))
# model.load_state_dict(state_dict)

Started dataset with 20599 words


In [216]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--max-epochs', type=int, default=10)
# parser.add_argument('--batch-size', type=int, default=256)
# parser.add_argument('--sequence-length', type=int, default=4)
# args = parser.parse_args()

args = {
    'max_epochs' : 5,
    'batch_size' : 256,
    'sequence_length' : 5,
    'max_len' : 100,
    
}

dataset = Dataset(tokenized,**args)
model = Model(dataset)

train(dataset, model, args)

model.index_to_word = dataset.index_to_word
model.word_to_index = dataset.word_to_index
from config import DATA_DIR
save_data(model, DATA_DIR / 'models' / 'massage_model.pickle')
# print(predict(dataset, model, text='בוקר טוב'))

Started dataset with 19804 words
{'epoch': 0, 'batch': 0, 'loss': 9.90282858972964}
{'epoch': 0, 'batch': 1, 'loss': 9.847893409729004}
{'epoch': 0, 'batch': 2, 'loss': 9.811496688098442}
{'epoch': 0, 'batch': 3, 'loss': 9.742628574371338}
{'epoch': 0, 'batch': 4, 'loss': 9.67920649781519}
{'epoch': 0, 'batch': 5, 'loss': 9.39333903271219}
{'epoch': 0, 'batch': 6, 'loss': 9.123680298145}
{'epoch': 0, 'batch': 7, 'loss': 8.881839243570964}
{'epoch': 0, 'batch': 8, 'loss': 7.736476961771647}
{'epoch': 0, 'batch': 9, 'loss': 7.177057013367161}
{'epoch': 0, 'batch': 10, 'loss': 7.343404524466571}
{'epoch': 0, 'batch': 11, 'loss': 6.923377905136499}
{'epoch': 0, 'batch': 12, 'loss': 6.802384463223544}
{'epoch': 0, 'batch': 13, 'loss': 8.568886969089508}
{'epoch': 0, 'batch': 14, 'loss': 7.578149857059602}
{'epoch': 0, 'batch': 15, 'loss': 6.82056043364785}
{'epoch': 0, 'batch': 16, 'loss': 7.446205753268617}
{'epoch': 0, 'batch': 17, 'loss': 6.219143558950985}
{'epoch': 0, 'batch': 18, 'los

{'epoch': 0, 'batch': 154, 'loss': 9.480313283205032}
{'epoch': 0, 'batch': 155, 'loss': 7.1690655118889275}
{'epoch': 0, 'batch': 156, 'loss': 7.0494432564704645}
{'epoch': 1, 'batch': 0, 'loss': 6.736425348122915}
{'epoch': 1, 'batch': 1, 'loss': 7.799169836919519}
{'epoch': 1, 'batch': 2, 'loss': 6.023248320040495}
{'epoch': 1, 'batch': 3, 'loss': 6.879906428310107}
{'epoch': 1, 'batch': 4, 'loss': 6.093615572298726}
{'epoch': 1, 'batch': 5, 'loss': 6.564278423786163}
{'epoch': 1, 'batch': 6, 'loss': 6.934145259333181}
{'epoch': 1, 'batch': 7, 'loss': 6.86876992046833}
{'epoch': 1, 'batch': 8, 'loss': 7.865807265043259}
{'epoch': 1, 'batch': 9, 'loss': 6.78135846555233}
{'epoch': 1, 'batch': 10, 'loss': 6.068089206372538}
{'epoch': 1, 'batch': 11, 'loss': 6.209139006005393}
{'epoch': 1, 'batch': 12, 'loss': 6.786198774550823}
{'epoch': 1, 'batch': 13, 'loss': 6.595889464020729}
{'epoch': 1, 'batch': 14, 'loss': 6.620175492949784}
{'epoch': 1, 'batch': 15, 'loss': 6.449136141574744}


{'epoch': 1, 'batch': 151, 'loss': 7.2817531398364475}
{'epoch': 1, 'batch': 152, 'loss': 6.505248798102867}
{'epoch': 1, 'batch': 153, 'loss': 6.013849872350693}
{'epoch': 1, 'batch': 154, 'loss': 6.420297361910343}
{'epoch': 1, 'batch': 155, 'loss': 5.457633541896939}
{'epoch': 1, 'batch': 156, 'loss': 6.468922401016409}
{'epoch': 2, 'batch': 0, 'loss': 6.61179686240528}
{'epoch': 2, 'batch': 1, 'loss': 7.132510058283806}
{'epoch': 2, 'batch': 2, 'loss': 6.342511171644384}
{'epoch': 2, 'batch': 3, 'loss': 5.655349674678984}
{'epoch': 2, 'batch': 4, 'loss': 5.326683563739062}
{'epoch': 2, 'batch': 5, 'loss': 6.523121844638478}
{'epoch': 2, 'batch': 6, 'loss': 6.206811929902723}
{'epoch': 2, 'batch': 7, 'loss': 6.565343491542033}
{'epoch': 2, 'batch': 8, 'loss': 5.648979736698998}
{'epoch': 2, 'batch': 9, 'loss': 5.6947634884944325}
{'epoch': 2, 'batch': 10, 'loss': 6.138639569282532}
{'epoch': 2, 'batch': 11, 'loss': 6.210609896315469}
{'epoch': 2, 'batch': 12, 'loss': 5.9596410046021

{'epoch': 2, 'batch': 148, 'loss': 6.302401720456881}
{'epoch': 2, 'batch': 149, 'loss': 5.929247143941048}
{'epoch': 2, 'batch': 150, 'loss': 6.598128976339989}
{'epoch': 2, 'batch': 151, 'loss': 6.188143886625767}
{'epoch': 2, 'batch': 152, 'loss': 5.7164301214546995}
{'epoch': 2, 'batch': 153, 'loss': 6.218376457691193}
{'epoch': 2, 'batch': 154, 'loss': 6.452031966298819}
{'epoch': 2, 'batch': 155, 'loss': 7.254565854668617}
{'epoch': 2, 'batch': 156, 'loss': 4.937644285815103}
{'epoch': 3, 'batch': 0, 'loss': 6.449802505664336}
{'epoch': 3, 'batch': 1, 'loss': 6.326954704649905}
{'epoch': 3, 'batch': 2, 'loss': 6.228745401915857}
{'epoch': 3, 'batch': 3, 'loss': 6.1486926641729145}
{'epoch': 3, 'batch': 4, 'loss': 5.015939436460796}
{'epoch': 3, 'batch': 5, 'loss': 6.245213650573383}
{'epoch': 3, 'batch': 6, 'loss': 6.02762903733687}
{'epoch': 3, 'batch': 7, 'loss': 6.1018826471020775}
{'epoch': 3, 'batch': 8, 'loss': 6.232789767365301}
{'epoch': 3, 'batch': 9, 'loss': 5.984186063

{'epoch': 3, 'batch': 145, 'loss': 5.901253215037286}
{'epoch': 3, 'batch': 146, 'loss': 5.963694536324703}
{'epoch': 3, 'batch': 147, 'loss': 5.9863065140587945}
{'epoch': 3, 'batch': 148, 'loss': 6.712664126753807}
{'epoch': 3, 'batch': 149, 'loss': 5.861751768833551}
{'epoch': 3, 'batch': 150, 'loss': 5.5625988682111105}
{'epoch': 3, 'batch': 151, 'loss': 5.945202021598816}
{'epoch': 3, 'batch': 152, 'loss': 5.558481979370117}
{'epoch': 3, 'batch': 153, 'loss': 5.649762981534004}
{'epoch': 3, 'batch': 154, 'loss': 6.209372844547033}
{'epoch': 3, 'batch': 155, 'loss': 5.939239527170475}
{'epoch': 3, 'batch': 156, 'loss': 5.742474045508947}
{'epoch': 4, 'batch': 0, 'loss': 5.591106641751069}
{'epoch': 4, 'batch': 1, 'loss': 5.883821133375168}
{'epoch': 4, 'batch': 2, 'loss': 5.614255056381226}
{'epoch': 4, 'batch': 3, 'loss': 5.789295080304146}
{'epoch': 4, 'batch': 4, 'loss': 5.812746364623308}
{'epoch': 4, 'batch': 5, 'loss': 5.851970965862274}
{'epoch': 4, 'batch': 6, 'loss': 5.695

{'epoch': 4, 'batch': 142, 'loss': 6.677961248385755}
{'epoch': 4, 'batch': 143, 'loss': 6.045738108158112}
{'epoch': 4, 'batch': 144, 'loss': 6.935833567664737}
{'epoch': 4, 'batch': 145, 'loss': 5.432217635490276}
{'epoch': 4, 'batch': 146, 'loss': 6.009946832785735}
{'epoch': 4, 'batch': 147, 'loss': 5.961039961995305}
{'epoch': 4, 'batch': 148, 'loss': 5.741732094647749}
{'epoch': 4, 'batch': 149, 'loss': 5.592143531198855}
{'epoch': 4, 'batch': 150, 'loss': 5.828634882966678}
{'epoch': 4, 'batch': 151, 'loss': 5.3237145960330965}
{'epoch': 4, 'batch': 152, 'loss': 6.426858616576475}
{'epoch': 4, 'batch': 153, 'loss': 6.365191072225571}
{'epoch': 4, 'batch': 154, 'loss': 5.916288710557497}
{'epoch': 4, 'batch': 155, 'loss': 5.786670856403582}
{'epoch': 4, 'batch': 156, 'loss': 5.6652305126190186}


In [217]:
# from apps.modeling.model import Model
# from apps.modeling.dataset import Dataset
# from extras.nlp.tokenizer import tokenize_text
# text = chat['text']
# tokenized_text = text.apply(tokenize_text)
# model = Model(Dataset(tokenized_text))
# model.load_state_dict(state_dict)

In [218]:
# model = Model(dataset)

def predict(model, text, sequence_length=5, next_words=100):
    model.eval()

    words = text.split(' ')
    start = 0
    if len(words) < sequence_length:
        start = sequence_length - len(words)
        words = [START_TOKEN] * (sequence_length - len(words)) + words
    words = words[-sequence_length:]
    
    for i in range(0, next_words):
        x = torch.tensor([[model.word_to_index[w] for w in words[-sequence_length:]]])
        y_pred = model(x)#, randomize = True)

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(model.index_to_word[word_index])
        if words[-1] == END_TOKEN:
            return words[start:-1]

    return words

def complete_sentence(model, sentence):
    answers = []
    for i in range(10):
        answers.append(' '.join(predict(model, text=sentence)))
    return answers

model = read_data(DATA_DIR / 'models' / 'massage_model.pickle')
complete_sentence(model, 'בוקר טוב')

['בוקר טוב שהדירות ממך לי 5 ון לשם',
 'בוקר טוב',
 'בוקר טוב מה בו לך !',
 'בוקר טוב ❤',
 'בוקר טוב ❤ את מפה ? . ז בערב בלא',
 'בוקר טוב',
 'בוקר טוב שבוע ❤',
 'בוקר טוב',
 'בוקר טוב',
 'בוקר טוב דווקא']

In [203]:
state_dict = model.state_dict()

מה נשמע הראלי וזהו
